<a href="https://colab.research.google.com/github/MK316/workspace/blob/main/Kangkim23/Step1_dataprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [1] Setting: openai install

In [ ]:
%%capture
!pip install openai

In [ ]:
import openai
openai.api_key = input('openai API key here')

[2] Google speech file to read and data summary
Jump to [1A]





In [ ]:

import pandas as pd

url = "https://raw.githubusercontent.com/MK316/workspace/main/Kangkim23/koreanaccent_log.csv"
df = pd.read_csv(url)
df.head()

In [ ]:
#@markdown Add a column 'Filename' korean03.mp3
file = df['L1']
nameid = df['Fileid']

newfilename = []

for i in range(0, len(file)):
  id = str(nameid[i])
  # if len(id) == 1:
  #   newid = "0" + id
  # else:
  #   newid = id
  new = file[i].strip() + id+".mp3"
  newfilename.append(new)

print(newfilename)
df['Filename'] = newfilename
df.head()

In [ ]:
df.to_csv('koreanaccent_info3.csv', index=False)

## [2A] Audio files from Google to Colab 'myaudio' directory

In [ ]:
!pwd

In [ ]:

#@markdown 🎯Mount Google drive and list files (e.g., "asrdata" folder in my case)
from google.colab import drive
import os

drive.mount('/content/drive')

mydir = input("Type the file directory in your google drive: e.g., asrdata  ")
!ls "/content/drive/MyDrive/{mydir}"
!pwd


*Specify zip file folder name on Google

In [ ]:
#@markdown 🎯 To do

#@markdown   [1] Make a new folder: type a new folder name (e.g., myaudio)
import os

folder_name = input("Type a name for the new folder.")

if not os.path.exists(folder_name):
  os.makedirs(folder_name)
  print(f"A new folder name '{folder_name}' has been created.")
else:
  print(f"{folder_name} already exists.")

#@markdown [2] Unzip files: type a zip file name (e.g., SE.zip)

zipname = input("Type your zip file name (e.g., se.zip) to process (unzip and save them under the new folder")
!unzip "/content/drive/MyDrive/accentdata/{zipname}" -d "/content/{folder_name}/"

print(f"Your {zipname} is unzipped under '{folder_name}' folder")

In [ ]:
#@markdown 🎯 Unmount Google drive (clearing): optional

from google.colab import drive
drive.flush_and_unmount()

# [1] Setting: openai install

In [ ]:
%%capture
!pip install openai

In [ ]:
import openai
openai.api_key = input('openai API key here')

### Note: Reading audio from github
```
import os
url = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/audio/F01_post.wav"
os.system("curl " + url + " > test.wav")
```

Note: file list of a colab directory
```
import os
path = '/content/myaudio/'
files = os.listdir(path)

for file in files:
    print(file)
```

In [ ]:
os.chdir("/content/myaudio")

In [ ]:
# A single file process to check
file = df['Filename'][10]
audio_file = open(file, "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
print(transcript['text'])

In [ ]:
file = df['Filename'][10]
audio_file = open(file, "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file, language="en")
print(transcript['text'])


In [ ]:
#@markdown 🎯Install {autotime} to measure runtime (From here, runtime appears automatically.)
%%capture
!pip install ipython-autotime
%load_ext autotime

In [ ]:
# os.chdir('/content/myaudio')
!pwd

In [ ]:
recog = []

for i in range(0, len(df['Fileid'])):
  file = df['Filename'][i]
  audio_file = open(file, "rb")
  transcript = openai.Audio.transcribe("whisper-1", audio_file, language="en")
  recog.append(transcript['text'])
print(recog)

In [ ]:
df['Recognized'] = recog
df.head()

In [ ]:
# Comparison between target and recognized
se = df

In [ ]:
#@markdown Calculate recognition rate, record missing words => dataframe (df2)
from nltk.tokenize import RegexpTokenizer

# number of words in the sentence
nws = []
# number of words in the recognized text
nwr = []
# number of missing words
nmw = []
# number of words actually recognized
nwar = []

# Recgonition Rate
rr = []
#
nr = []
# Missing word list
mw = []
# Correctly recognized word list
recword=[]


for i in range(0,len(se['Target'])):
  t1 = se['Target'][i]

# text 1
  txt1 = t1.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist = tokenizer.tokenize(txt1)
  nt = len(wlist)
  nws.append(nt)

# text 2
  t2 = se['Recognized'][i]
  txt2 = t2.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist1 = tokenizer.tokenize(txt2)
  nt1 = len(wlist1)
  nwr.append(nt1)

# Recognition rate

  # from tables.idxutils import calc_chunksize
  # from nltk.downloader import ErrorMessage
# txt1(original text), txt2 (recognized text)

  mword = []
  rword = []
  score = 0
  for i in range(0, len(wlist1)):
      w = wlist1[i]

      if w in wlist:
        sc = 1
        rword.append(w)
      else:
        sc = 0
        mword.append(w)

      score = score + sc
      mwords = ', '.join(mword)
      rwords = ', '.join(rword)
  mw.append(mwords)
  missingword = round((score/len(wlist))*100,2)
  nr.append(score)
  recword.append(rwords)

  # RecRate = float(format(missingword, '.2f'))
  # ErrRate = float(format((100.0 - RecRate), '.2f'))

  rr.append(missingword)
  # print('Matching words: %d'%score, 'out of %d words'%len(wlist))
  # print("="*50)
  # print('Recognition Rate: %f %%'%RecRate)
  # print('Error Rate: %f %%'%ErrRate)

se['LenS'] = nws
se['LenR'] = nwr
se['N_RecW'] = nr
se['RecRate'] = rr
se['Recognized_words'] = recword
se['MissRec_words'] = mw

se.head(); se.tail()

In [ ]:
se.describe()

In [ ]:
os.chdir('/content')

In [ ]:
df1 = se
df1.to_csv("koreanaccent_recognized.csv", index=False)

# 💜 From here: df (with recognized words and texts)

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/workspace/main/Kangkim23/koreanaccent_recognized.csv"
df = pd.read_csv(url)
df.head()

In [ ]:
data = df[['Filename', 'Age', 'L2Onset','LOR','LenR','N_RecW','RecRate']]

# WER

In [ ]:
#@markdown + Install and import libraries
%%capture
!pip install Levenshtein
import Levenshtein as lev
import numpy as np
import pandas as pd

In [ ]:
#@markdown + Calculate WER and WER_lev and add columns to df
# import numpy as np
# import pandas as pd
# import Levenshtein as lev
#@markdown + Column names 'Original' and 'Recognized' for comparison
def calculate_wer_word_level(s1, s2):
    """
    Calculation of WER with Dynamic Time Warping algorithm at word level.
    """

    # splitting the sentences into words
    s1_words = s1.split()
    s2_words = s2.split()

    # preparing the matrix for the Dynamic Time Warping algorithm
    dtw_matrix = np.zeros((len(s1_words)+1, len(s2_words)+1))
    for i in range(len(s1_words)+1):
        for j in range(len(s2_words)+1):
            if i == 0:
                dtw_matrix[0][j] = j
            elif j == 0:
                dtw_matrix[i][0] = i

    # Dynamic Time Warping algorithm
    for i in range(1, len(s1_words)+1):
        for j in range(1, len(s2_words)+1):
            cost = 0 if s1_words[i-1] == s2_words[j-1] else 1
            dtw_matrix[i][j] = cost + min([dtw_matrix[i-1][j], dtw_matrix[i][j-1], dtw_matrix[i-1][j-1]])

    wer = dtw_matrix[len(s1_words)][len(s2_words)] / len(s1_words)

    return wer

def calculate_wer_lev(s1, s2):
    """
    Calculation of WER with Levenshtein distance at character level.
    """

    # splitting the sentences into words
    s1_words = s1.split()
    s2_words = s2.split()

    # calculating Levenshtein distance
    edit_distance = lev.distance(' '.join(s1_words), ' '.join(s2_words))

    # normalizing by the length of the original sentence (s1)
    wer = edit_distance / len(s1_words)

    return wer


In [ ]:
# Assuming df is your DataFrame and it has columns 'Sentence' and 'Recognized'
import numpy as np

wer_values_word_level = df.apply(lambda row: calculate_wer_word_level(row['Target'], row['Recognized']), axis=1)
wer_values_lev = df.apply(lambda row: calculate_wer_lev(row['Target'], row['Recognized']), axis=1)

# Adding WER values as new columns to the DataFrame
df['WER'] = wer_values_word_level
df['WER_lev'] = wer_values_lev

In [ ]:
df1 = df[['Filename', 'Age', 'L2Onset','LOR','LenR','N_RecW','RecRate','WER','WER_lev']]
df1

In [ ]:
https://github.com/MK316/workspace/tree/main/ASR03/audio

In [ ]:
os.chdir('/content')

In [ ]:
import os
url = "https://raw.githubusercontent.com/MK316/workspace/main/ASR03/audio/korean30.mp3"
os.system("curl " + url + " > test.mp3")


In [ ]:
audio_file = open('test.mp3', "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
print(transcript['text'])

In [ ]:
df.to_csv('ASR03_results.csv',index=False)